# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [167]:
import requests
from bs4 import BeautifulSoup
from IPython.display import display, HTML
import re
from datetime import datetime as dt
from datetime import timedelta as td
import pytz
import time
import pandas as pd


def convert_to_utc(date_str, now, tz):
    '''
    Форматы:
        13 августа 2020 в 12:07
        сегодня в 11:19
        вчера в 23:29    
    '''
    t = None
    for day in ["сегодня", "вчера"]:
        if day in date_str:
            res = re.findall(r"(\d\d):(\d\d)",  date_str)
            if res is not None and len(res) == 1 and len(res[0]) == 2:
                if day == "вчера":
                    t = now.replace(hour=int(res[0][0]), minute=int(res[0][1]), second=0) + td(days=-1)
                else:
                    t = now.replace(hour=int(res[0][0]), minute=int(res[0][1]), second=0)
            break
        
    if t is None:
        res = re.findall(r"(\d{1,2})\s(\w+)\s(\d{4}) в (\d\d):(\d\d)", date_str)
        if res is not None and len(res) == 1 and len(res[0]) == 5:
            months = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
              'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря']
            if res[0][1] in months:
                month = months.index(res[0][1]) + 1
                t = dt(int(res[0][2]), months.index(res[0][1]) + 1, int(res[0][0]),
                    int(res[0][3]), int(res[0][4]), tzinfo=tz)
        
    if t is not None:
        return t.astimezone(pytz.timezone("UTC")).strftime("%Y-%m-%d %H:%M:%S")
    
def get_table1(keywords):
    # инициализация
    info = []
    tz = pytz.timezone("Europe/Moscow")
    now = dt.now(tz)
    rex = '(' + '|'.join(keywords) + ')'

    # получаем страницу с самыми свежими постами
    req = requests.get('https://habr.com/ru/all/')
    soup = BeautifulSoup(req.text, 'html.parser')

    # извлекаем посты и ищем
    posts = soup.find_all('article', class_='post')
    for post in posts:
        post_id = post.parent.attrs.get('id')
        if not post_id:
            continue

        # определяем дату/время, заголовок и ссылку
        time_element = post.find('span', class_='post__time')
        time = convert_to_utc(time_element.text, now, tz)
        title_element = post.find('a', class_='post__title_link')
        header = title_element.text
        link = title_element.attrs.get('href')

        # ищем в хабах
        found = None
        hubs = post.find_all('a', class_='hub-link')
        for hub in hubs:        
            if len(re.findall(rex, hub.text, flags=re.IGNORECASE)):
                found = 'хабы'
                break

        # ищем в заголовке
        if found is None:
            if len(re.findall(rex, header, flags=re.IGNORECASE)):
                found = 'заголовок'

        # ищем в тексте
        if found is None:
            post_text = post.find('div', class_='post__text').text
            if len(re.findall(rex, post_text, flags=re.IGNORECASE)):
                found = 'текст'
        
        info.append([time, header, link, found])
            
    return info

def get_table2(pubs, keywords):
    rex = '(' + '|'.join(keywords) + ')'
    info = []
    
    for pub in pubs:
        # получаем страницу с самыми свежими постами
        req = requests.get(pub[2])
        soup = BeautifulSoup(req.text, 'html.parser')
        
        post_text = soup.find('div', class_='post__text').text

        # ищем в тегах
        found = pub[3]
        if found is None:
            post_tags = soup.find('dl', class_='post__tags')
            tags = post_tags.find_all('a', class_='post__tag')
            for tag in tags:        
                if len(re.findall(rex, tag.text, flags=re.IGNORECASE)):
                    found = 'тэги'
                    break
        
        # ищем в тексте
        if found is None:
            if len(re.findall(rex, post_text, flags=re.IGNORECASE)):
                found = 'весь текст'
        
        # добавляем только нужные строки
        if found is not None:
            info.append(pub[:3] + [found, post_text])
            
        time.sleep(0.5)
            
    return info 


KEYWORDS = ['python', 'парсинг']

table1 = get_table1(KEYWORDS)
table2 = get_table2(table1, KEYWORDS)

pdt1 = pd.DataFrame(table1, columns=["Дата", "Заголовок", "Ссылка", "Найден"]).dropna()
display(HTML(pdt1.iloc[:, [0,1,2]].to_html()))

pdt2 = pd.DataFrame(table2, columns=["Дата", "Заголовок", "Ссылка", "Найден", "Текст статьи"])
display(HTML(pdt2.iloc[:, [0,1,2,4]].to_html()))

,Дата,Заголовок,Ссылка
7,2020-08-16 14:50:00,Конфуций и Маргарита,https://habr.com/ru/post/515346/
12,2020-08-16 13:03:00,VKWave — фреймворк для разработки ботов ВКонтакте,https://habr.com/ru/post/515334/
17,2020-08-16 11:55:00,Первые шаги в визуализации данных с использованием Geopandas и OSM,https://habr.com/ru/post/515328/


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [221]:
def get_email_info(email):
    headers = {
        "Accept": "application/json, text/plain, */*",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Content-Length": "32",
        "Content-Type": "application/json;charset=UTF-8",
        "Host": "digibody.avast.com",
        "Origin": "https://www.avast.com",
        "Pragma": "no-cache",
        "Referer": "https://www.avast.com/hackcheck",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-site",
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36"
    }
    
    time.sleep(0.5)
    req = requests.post("https://digibody.avast.com/v1/web/leaks", data=f'{{"email": "{email}"}}', headers=headers)
    return req.json()

def fix_date(val):
    date = dt.fromtimestamp(int(val) // 1000)
    return date.strftime("%Y-%m-%d")

def check_emails(emails):
    table = []
    for email in emails:
        resp = get_email_info(email)
        for leak in resp['value']:
            table.append([email, fix_date(leak['leak_info']['date']), leak['leak_info']['title'], leak['leak_info']['description']])
    return table

EMAIL = ["afilis2005@yandex.ru"]

table = check_emails(EMAIL)
pdt = pd.DataFrame(table, columns=["Почта", "Дата утечки", "Источник утечки", "Описание утечки"])
display(HTML(pdt.to_html()))

,Почта,Дата утечки,Источник утечки,Описание утечки
0,afilis2005@yandex.ru,2019-05-23,LiveJournal,"In 2017, social network LiveJournal's database was allegedly breached. The stolen data contains passwords, email addresses and additional personal information. This breach is being privately shared in the dark web."
1,afilis2005@yandex.ru,2019-02-06,Collection #5 Combo List,"On January 7, 2019, an online user named Sanixer advertised 5 massive collections of leaked email and password combinations for sale on a popular hacking forum. These collections include a total of 993GB of personal information and seem to be mostly rehashes of previous breaches. The sheer size of so much breach data available in one place captured a lot of attention in the mainstream media. \r\n\r\nThe collections were hosted for a short time on the popular hosting service MEGA, and have since been shared via download torrents and private links hundreds of times. Lifetime access to the data was being sold for $45.\r\n\r\nCollection #5 consists of more than 540 million unique records across 16,022 individual files."
2,afilis2005@yandex.ru,2019-02-06,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanixer advertised 5 massive collections of leaked email and password combinations for sale on a popular hacking forum. These collections include a total of 993GB of personal information and seem to be mostly rehashes of previous breaches. The sheer size of so much breach data available in one place captured a lot of attention in the mainstream media. \r\n\r\nThe collections were hosted for a short time on the popular hosting service MEGA, and have since been shared via download torrents and private links hundreds of times. Lifetime access to the data was being sold for $45.\r\n\r\nThe ""Antipublic #1"" collection consists of more than 1.7 billion unique records across 303 individual files."
3,afilis2005@yandex.ru,2019-01-29,Collection #2 Combo List,"On January 7, 2019, an online user named Sanixer advertised 5 massive collections of leaked email and password combinations for sale on a popular hacking forum. These collections include a total of 993GB of personal information and seem to be mostly rehashes of previous breaches. The sheer size of so much breach data available in one place captured a lot of attention in the mainstream media. \r\n\r\nThe collections were hosted for a short time on the popular hosting service MEGA, and have since been shared via download torrents and private links hundreds of times. Lifetime access to the data was being sold for $45.\r\n\r\nCollection #2 consists of more than 3 billion unique records across 24,921 individual files."
4,afilis2005@yandex.ru,2020-03-05,Sensitive Source,This source has been marked as sensitive due to one of the following reasons: Revealing the source may compromise an on-going investigation. The affected site is of a controversial nature but does not validate email addresses and could therefore be used to tarnish an employee's reputation.
5,afilis2005@yandex.ru,2019-12-13,Sensitive Source,This source has been marked as sensitive due to one of the following reasons: Revealing the source may compromise an on-going investigation. The affected site is of a controversial nature but does not validate email addresses and could therefore be used to tarnish an employee's reputation.
6,afilis2005@yandex.ru,2017-06-14,NNM-Club,"In September 2013, Russian torrent tracker NNM-Club's user database was allegedly breached. The stolen data contains around 7 million records including email addresses, usernames, and passwords. On January 2014, the data was shared on publicly available cybercrime and forums."


In [21]:
NEWSFEED_REQUEST = 'https://api.vk.com/method/wall.get?'
TOKEN = ''
VERSION = '5.103'
SLEEP = 0.33

params = {
    'access_token': TOKEN,
    'v': VERSION,
    'owner_id': -30159897,
    'count': 50,
}

def change_date(val):
    date = dt.fromtimestamp(val)
    return date.strftime("%Y-%m-%d %H:%M")


res = requests.get(NEWSFEED_REQUEST, params)
resps = res.json()
posts = []
for item in resps['response']['items']:
    posts.append([change_date(item['date']), item['text']])

fr = pd.DataFrame(posts, columns=["Дата поста", "Текст поста"]).sort_values("Дата поста", ascending=False)
display(HTML(fr.to_html()))

,Дата поста,Текст поста
1,2020-08-17 19:38,"Зарядку сделали, перед зеркалом порепетировали и даже всех слушателей голыми представили🙃 А страх публичных выступлений всё равно остался.\n\nРазбираемся, как самостоятельно это исправить → http://netolo.gy/fLf"
2,2020-08-17 13:00,"Как приятно видеть, что работа приносит результат: конверсия на запись с вашей рекламной кампании составила больше 40%, это очень высокий показатель👍🏻. А вас уже ждёт новая должность — вы теперь продюсер! Курс пользуется популярностью, студентов всё больше, а руководство строит смелые планы по развитию направления. \n \nЧтобы продвигать курс дальше, нужны средства. Поэтому сегодня в вашем личном кабинете появилась новая задача — найти инвестора. Вам назначили встречу в крупном бизнес-центре. Готовы рассказать о своём курсе потенциальным спонсорам? \n\n‼ Чтобы задание засчиталось, вам необходимо выделить пять основных преимуществ вашего курса и перечислить их отдельной строкой. Например: \n \n📌— интерактивность; \n— простой язык; \n— свежая информация; \n— практическая польза; \n— востребованность📌\n \nТакже необходимо оставить любой комментарий под 3 комментариями других игроков.\n \n‼ Чтобы задание было засчитано, ваш ответ должен состоять минимум из 50 символов."
3,2020-08-17 11:42,"Запустили бесплатный курс по основам контекстной рекламы ⭐️\n\nВнутри — базовые знания, которые помогут шаг за шагом настроить рекламу в Яндекс.Директ:\n▪Курс создан специально для новичков: вы не потеряетесь в непонятных терминах\n▪Научитесь собирать семантику и настроите два типа рекламных кампаний \n▪А ещё поймёте, интересна ли вам работа специалиста по контекстной рекламе\n\nУчитесь в удобном темпе — материалы станут доступны сразу после регистрации → http://netolo.gy/fLb"
4,2020-08-16 16:28,"Подборка книг, которые помогут развить математическое мышление. \n\nДаже если школьная программа оставила лишь воспоминания о том, как сложно и скучно было на уроках математики 🙄 \n\n▪«Как не ошибаться. Сила математического мышления», Джордан Элленберг\nАвтор считает, что математика — это надстройка к здравому смыслу, которая значительно преумножает его возможности и силу. Он рассказывает о самых разных явлениях и идеях — от рейганомики, лотерейных схем и искусственных языков до развития неевклидовой геометрии, живописи итальянского Ренессанса и того, что Фейсбук может (и что не может) узнать о вас.\n\n▪«Думай как математик. Как решать любые проблемы быстрее и эффективнее», Барбара Оакли \nЭта книга не о сложных числах и уравнениях, а о тонкостях работы нашего мозга. И о том, как уравновесить знания в голове, решать задачи и превратить обучение в привычку. Оакли делится советами и задачами, о которых ей рассказали преподаватели и профессора со всего света. \n\n▪«Модельное мышление. Как анализировать сложные явления с помощью математических моделей», Скотт Пейдж\nКнига рассказывает, как решать проблемы и осмыслять сложные вопросы с помощью математических моделей. Автор в увлекательной форме описывает двадцать пять широких классов моделей (таких как модели роста, случайные блуждания, энтропия, цепи Маркова и многое другое), объясняя, как и когда их применять.\n\n▪«Математика жизни. Простые алгоритмы принятия верных решений», Брайан Кристиан и Том Гриффитс\nАвторы книги считают, что программисты и математики уже давно разработали алгоритмы, которые позволяют компьютерам найти оптимальное решение в заданное время и с минимальными затратами сил и средств. Книга популярно объясняет, как именно применять сложные математические алгоритмы для решения повседневных задач.\n\n▪«Вычислительное мышление. Метод решения сложных задач», Пол Керзон и Питер МакОуэн\nВ книге рассказывается об основных составляющих вычислительного мышления: алгоритмическом мышлении, декомпозиции, абстракции, логики. Авторы собрали игры и головоломки, которые помогут развить вычислительное мышление и научиться использовать его для решения повседневных проблем."
5,2020-08-16 12:30,"Продвижение курса — дело непростое. Но у 

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
